In [1]:
import hs_alkane.alkane as mdl
import numpy as np

# For progress bar
from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
import sys
import os


nwalkers = 500
active_box = nwalkers+1  #extra box to hold positions
nchains = 32
nbeads = 1
max_vol_per_atom = 15
mdl.box_set_num_boxes(nwalkers+1)
mdl.box_initialise()
mdl.box_set_pbc(1)
mdl.alkane_set_nchains(nchains) 
mdl.alkane_set_nbeads(nbeads)    
mdl.alkane_initialise()           
mdl.box_set_isotropic(1)
mdl.box_set_bypass_link_cells(1) # Bypass use of link cell algorithm for neighbour finding
mdl.box_set_use_verlet_list(0)   # Don't use Verlet lists either since CBMC moves quickly invalidate these

cell_matrix = 0.999*np.eye(3)*np.cbrt(nbeads*nchains*max_vol_per_atom)#*np.random.uniform(0,1)
for ibox in range(1,nwalkers+1):
    mdl.box_set_cell(ibox,cell_matrix)


    

In [2]:
from ase import Atoms
from ase.visualize import view
import ase

def mk_ase_config(ibox, Nbeads, Nchains):
    'Uses the current state of the alkane model to construct an ASE atoms object'
    
    # Create and populate ASE object
    model_positions = np.empty([Nchains*Nbeads, 3])
    cell_vectors = mdl.box_get_cell(ibox)
   
    for ichain in range(0, Nchains):
        model_positions[Nbeads*ichain:Nbeads*ichain+Nbeads] = mdl.alkane_get_chain(ichain+1, ibox)
    
    confstring = "C"+str(Nbeads*Nchains)
    
    box_config = Atoms(confstring, positions=model_positions*(1.5/0.4), pbc=True, cell=cell_vectors*(1.5/0.4))


    return box_config  # Returns ASE atom object

In [3]:
def vis_chains(vis_config):
    '''Takes an ASE atoms object or list thereof and creates a customised ngl viewer 
       with appropriate settings for our bulk alkane chain models'''
    
    met = 0.35
    rad = 1.0
    
    colours = ['#DDDDDD']#['#FF1111','#FFAAAA', '#DDDDDD', '#1111FF', '#AAAAFF']
    ncols = len(colours)
    
    sel=list()
    for icol in range(ncols):
        sel.append(list())
    
    # Create lists for each colour
    for ichain in range(0, nchains):
        icol = ichain%ncols
        for ibead in range(nbeads):
            iatom = ichain*nbeads + ibead
            sel[icol].append(iatom)
            
    v = view(vis_config, viewer='ngl')                   
    v.view.clear_representations()
    v.view.add_representation('unitcell', color='#000000')
    
    for icol in range(ncols):
        v.view.add_representation('ball+stick', selection=sel[icol], color=colours[icol], radius=rad, metalness=met)

    return v

In [4]:
ncopy = nchains
for ibox in range(1,nwalkers+1):
    for ichain in range(1,ncopy+1):
        rb_factor = 0
        mdl.alkane_set_nchains(ichain)
        overlap_flag = 1
        while overlap_flag != 0 and ichain != 1:
            rb_factor, ifail = mdl.alkane_grow_chain(ichain,ibox,1)         
#             if ifail != 0:
#                 rb_factor = 0
            overlap_flag = mdl.alkane_check_chain_overlap(ibox)

        


In [23]:
overlap_check = np.zeros(nwalkers)
for ibox in range(1,nwalkers+1):
    overlap_check[mdl.alkane_check_chain_overlap(ibox)]
print(np.where(overlap_check!=0))
#Checks for overlaps in any of the walkers

(array([], dtype=int64),)


In [6]:
atoms = mk_ase_config(10, nbeads, nchains)
#atoms = ase.io.read("test.traj")
vis_chains(atoms)


In [7]:
atoms = mk_ase_config(5, nbeads, nchains)
#atoms = ase.io.read("test.traj")
vis_chains(atoms)

In [8]:
mdl.alkane_set_dr_max(0.65)
mdl.alkane_set_dt_max(0.43)
mdl.alkane_set_dh_max(0.4)
mdl.alkane_set_dv_max(0.5)

In [9]:
move_types = ['box','translate', 'rotate', 'dihedral']
ivol = 0; itrans = 1; irot = 2; idih = 3
move_ratio = np.zeros(4)
move_ratio[ivol] = 1
move_ratio[itrans] = nchains
#moves_ratio[irot] = 2.0*nchains
#moves_ratio[idih] = 1.0*nchains
#moves_prob = np.cumsum(moves_ratio)/np.sum(moves_ratio)
walk_length = 20


Function which performs the majority of Monte Carlo steps. Has an optional volume_limit argument which allows
one to specify whether or there is a volume restriction when making volume moves. If not specified, its taken as positive infinity.

In [10]:
def MC_run(sweeps, move_ratio, ibox,volume_limit=float('inf')):
    moves_accepted = np.zeros(4)
    moves_attempted = np.zeros(4)
    nbeads = mdl.alkane_get_nbeads()
    nchains = mdl.alkane_get_nchains()
    isweeps = 0
    pressure = 0
    move_prob = np.cumsum(move_ratio)/np.sum(move_ratio)
    while isweeps < sweeps:
        imove=0
        while imove< nchains+1:
            ichain = np.random.randint(1, high=nchains+1) # picks a chain at random
            current_chain = mdl.alkane_get_chain(ichain+1, ibox)
            backup_chain = current_chain.copy()
            xi = np.random.random()
            if xi < move_prob[ivol]:
                # Attempt a volume move
                itype = ivol
                boltz = mdl.alkane_box_resize(pressure, ibox, 0)
                moves_attempted[itype] += 1
            elif xi < move_prob[itrans]:
                # Attempt a translation move
                itype = itrans
                boltz = mdl.alkane_translate_chain(ichain+1, ibox)
                moves_attempted[itrans] += 1
            elif xi < move_prob[irot]:
                # Attempt a rotation move
                itype = irot
                boltz, quat = mdl.alkane_rotate_chain(ichain+1, ibox, 0)
                moves_attempted[itype] += 1
            else:
                # Attempt a dihedral angle move
                itype = idih
                boltz, bead1, angle = mdl.alkane_bond_rotate(ichain+1, ibox, 1)
                moves_attempted[itype] += 1

#             if (np.random.random() < boltz):
#                 #accept the move
#                 moves_accepted[itype]+=1

#             else:
                
#                 # Reject move
#                 if (itype!=ivol):
#                     # Restore old chain if single chain move
#                     for ibead in range(nbeads):
#                         current_chain[ibead] = backup_chain[ibead]
#                 else:
#                     # Reset the box change - special fucntion for this.
#                     dumboltz = mdl.alkane_box_resize(pressure, ibox, 1)
            
            if (itype==ivol):
                new_volume = mdl.box_compute_volume(ibox)
                if (np.random.random() < boltz) and (new_volume - volume_limit) < sys.float_info.epsilon:
                    moves_accepted[itype]+=1
                else:
                    dumboltz = mdl.alkane_box_resize(pressure, ibox, 1)

                
            else:
                if (np.random.random() < boltz):
                #accept the move
                    moves_accepted[itype]+=1
                else:
                    #reject the move
                    for ibead in range(nbeads):
                        current_chain[ibead] = backup_chain[ibead]

                    
            


            imove += 1
        isweeps +=1
    moves_attempted = np.where(moves_attempted == 0, 1, moves_attempted)
    moves_acceptance_rate = moves_accepted/moves_attempted

    return mdl.box_compute_volume(ibox), moves_acceptance_rate

In [11]:
volumes = {}
start_volumes = []
for ibox in range(1,nwalkers+1):
    volumes[ibox], rate = MC_run(walk_length, move_ratio, ibox)
    start_volumes.append(volumes[ibox])


Check for overlaps again

In [12]:
overlap_check = np.zeros(nwalkers)
for ibox in range(1,nwalkers+1):
    overlap_check[mdl.alkane_check_chain_overlap(ibox)]
print(np.where(overlap_check!=0))

(array([], dtype=int64),)


In [13]:
atoms = mk_ase_config(7, nbeads, nchains)
atoms.wrap()
# v=view(atoms, viewer = 'nglview')
# v.view.clear_representations()
# v.view.add_representation('ball+stick', radius = 1)
# v.view.add_representation('unitcell')
v = vis_chains(atoms)
v

In [14]:
#output all volumes

Functionality for cloning one box onto another

In [15]:
def clone_walker(ibox_original,ibox_clone):
    #print(mdl.box_get_cell(ibox_original))

    cell = mdl.box_get_cell(ibox_original)
    mdl.box_set_cell(ibox_clone,cell)
#     print(mdl.box_get_cell(ibox_clone))
    
    
    for ichain in range(1,nchains+1):
        original_chain = mdl.alkane_get_chain(ichain,ibox_original)
        original_chain_copy = original_chain.copy()
        clone_chain = mdl.alkane_get_chain(ichain,ibox_clone)
        for i in range(nbeads):
#             print(original_chain[i][:])
#             print(original_chain_copy[i][:])
            
            clone_chain[i][:] = original_chain_copy[i][:]
#            print(clone_chain[i][:])
            #this checks for nans
            coord_sum = np.sum(clone_chain[i][:])
            nan_present = np.isnan(coord_sum)
            if nan_present:
                print(clone_chain[i][:])
                sys.exit()

                
            

Test cloning walker code below

In [16]:
# first_config = mk_ase_config(1,1,32)
# first_config.wrap()
# vis_chains(first_config)

In [17]:
# clone_walker(1,10)
# second_config = mk_ase_config(10,1,32)
# second_config.wrap()
# vis_chains(second_config)

In [18]:
def adjust_dv(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [1,0,0,0]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[ivol]
    
    if r < lower_bound:
        mdl.alkane_set_dv_max(mdl.alkane_get_dr_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dv_max(mdl.alkane_get_dr_max()*equil_factor)
    return r


    

In [19]:
def adjust_dr(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [0,1,0,0]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[itrans]
    
    if r < lower_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dr_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dr_max()*equil_factor)
    return r

    

In [20]:
def adjust_dt(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [0,0,1,0]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[irot]
    
    if r < lower_bound:
        mdl.alkane_set_dt_max(mdl.alkane_get_dr_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dt_max(mdl.alkane_get_dr_max()*equil_factor)
    return r


    

In [21]:
def adjust_dh(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [0,0,0,1]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[idih]
    
    if r < lower_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dh_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dh_max()*equil_factor)
    return r



In [22]:
ns_iterations = 5000
walk_length = 50

mc_adjust_interval = nwalkers//2 #for adjusting step sizes
low_acc_rate = 0.2
high_acc_rate = 0.5

removed_volumes = []

snapshots = 200

vis_interval = ns_iterations//snapshots


f = IntProgress(min=0, max=ns_iterations) 
display(f) # display the bar


try:
    os.remove("nestedsampling.traj")
except:
    pass

traj = ase.io.Trajectory("hsa_ns.traj",'a')

for i in range(ns_iterations):
    index_max = max(volumes, key=volumes.get)
    volume_limit = volumes[index_max]
    box_to_clone = np.random.randint(1,nwalkers+1)
    
    if ns_iterations%vis_interval == 0:
        #print("something")
        traj.write(atoms = mk_ase_config(index_max, nbeads, nchains))

    

    
    if ns_iterations%mc_adjust_interval == 0:
        print(volumes[index_max])

        adjust_dv(box_to_clone,active_box,low_acc_rate,high_acc_rate)
        r = adjust_dr(box_to_clone,active_box,low_acc_rate,high_acc_rate)
#         adjust_dt(clone,active_box,low_acc_rate,high_acc_rate)
#         adjust_dh(clone,active_box,low_acc_rate,high_acc_rate)
    
    clone_walker(box_to_clone,active_box) #copies the ibox from first argument onto the second one.
    
    new_volume,_ = MC_run(walk_length, move_ratio, active_box)
    
    
#     if new_volume < volume_limit:
    if True:
        removed_volumes.append(volume_limit)
        volumes[index_max] = new_volume #replacing the highest volume walker
        clone_walker(active_box, index_max)
        

        
    
    

    f.value+=1
traj.close()

IntProgress(value=0, max=5000)

481.9405963600425
[[7.83125169 0.         0.        ]
 [0.         7.83125169 0.        ]
 [0.         0.         7.83125169]]
[[7.83125169 0.         0.        ]
 [0.         7.83125169 0.        ]
 [0.         0.         7.83125169]]
[[7.83125169 0.         0.        ]
 [0.         7.83125169 0.        ]
 [0.         0.         7.83125169]]
[[7.80571027 0.         0.        ]
 [0.         7.80571027 0.        ]
 [0.         0.         7.80571027]]
481.68893349746827
[[7.82707852 0.         0.        ]
 [0.         7.82707852 0.        ]
 [0.         0.         7.82707852]]
[[7.82707852 0.         0.        ]
 [0.         7.82707852 0.        ]
 [0.         0.         7.82707852]]
[[7.82707852 0.         0.        ]
 [0.         7.82707852 0.        ]
 [0.         0.         7.82707852]]
[[7.81982732 0.         0.        ]
 [0.         7.81982732 0.        ]
 [0.         0.         7.81982732]]
481.5806619143095
[[7.83328019 0.         0.        ]
 [0.         7.83328019 0.        ]
 

SystemExit: 

/home/chem/msunfs/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
final_volumes = []
for ibox in range(1,nwalkers+1):
    final_volumes.append(volumes[ibox])
print(final_volumes)

In [ ]:
plt.hist(start_volumes, bins = 100)
plt.hist(final_volumes, bins = 100)
plt.show()

Two approaches to cloning a simulation cell for generating new configurations.

1. Write a random configuration to an xyz file, and then load it into an extra empty walker, which will be used to perform trial moves, and then save as an xyz file. Then you can use the highest volume actual walker to load the positions, and continue. Issue is, loading and saving will be the brunt of the calculation.

2. Use a numpy array to hold positions (i.e use it as a backup), and then manually change one of the walkers to be the clone, by setting positions and unit cell. Can get very messy, but should be much faster.

3. Blend the methods. Use an extra walker to hold the atoms but enter the positions and cell vectors from numpy array

In [ ]:
overlap_check = np.zeros(nwalkers)
for ibox in range(1,nwalkers+1):
    overlap_check[mdl.alkane_check_chain_overlap(ibox)]
print(np.where(overlap_check!=0))
#Checks for overlaps in any of the walkers

In [ ]:
atoms = mk_ase_config(6, nbeads, nchains)
atoms.wrap()
v = vis_chains(atoms)
#v = view(atoms,viewer='nglview')
v


In [ ]:
atoms_traj = ase.io.read("hsa_ns.traj", index = ':')

vis_chains(atoms_traj)

In [ ]:
traj.close()

In [ ]:
atoms_traj

In [ ]:
#for rdf stuff
def py_rdf(atoms, dr, dim):
    from numpy import zeros, sqrt, where, pi, mean, arange, histogram, absolute
    r = atoms.positions
    S = np.amax(atoms.cell)
    num_particles  = len(r)
    rMax           = S/2.0
    edges          = arange(0, rMax + dr, dr)
    num_increments = len(edges) - 1
    g              = zeros(num_increments)
    radii          = zeros(num_increments)
    numberDensity  = len(r) / atoms.get_volume()

    # Compute pairwise correlation for each particle
    for index in range(num_particles):

        d = 0.0
        for i in range(dim):
            dp = absolute(r[index,i] - r[:,i])
            mask = dp>S/2.0
            dp[mask] = S - dp[mask]
            d += dp*dp

        d = sqrt(d)
        d[index] = 2 * rMax

        (result, bins) = histogram(d, bins=edges, density=False)
        g += result

    g = g/(num_particles * numberDensity)

    # Normalize the g(r) dividing by the g(r) of an ideal gas - in 2D!
    if dim == 2:
        for i in range(num_increments):
            radii[i] = (edges[i] + edges[i+1]) / 2.
            rOuter = edges[i + 1]
            rInner = edges[i]
            g[i] = g[i] / (2.0 * pi * (rOuter-rInner)* radii[i])
    
    # Needed to compute the 3D g(r) (blue box)
    # Normalize the g(r) divinding by the g(r) of an ideal gas - in 3D!
    if dim == 3:
        for i in range(num_increments):
            radii[i] = (edges[i] + edges[i+1]) / 2.
            rOuter = edges[i + 1]
            rInner = edges[i]
            g[i] = g[i] / (4.0 * pi * (rOuter-rInner)* radii[i] * radii[i] )

    return (radii, g)

In [ ]:
r = atoms.positions
S = 20
dim = 3
dr = 0.5


radii, g = py_rdf(atoms, dr, dim)

In [ ]:
plt.plot(radii,g)
plt.show()

In [ ]:
np.trapz(g,radii)